In [8]:
import os
import pandas as pd

def process_excel_files(directory, output_directory, ssp, period):
    combined_data = {}

    for filename in os.listdir(directory):
        if filename.endswith(".xlsx") and ssp in filename and period in filename:
            file_path = os.path.join(directory, filename)
            df = pd.read_excel(file_path)

            # Get available return periods dynamically
            return_periods = [int(col.split('-year')[0].split()[-1]) for col in df.columns if 'Return value for' in col]

            for catchment_id in df['Catchment ID'].unique():
                catchment_data = df[df['Catchment ID'] == catchment_id]
                
                if catchment_id not in combined_data:
                    combined_data[catchment_id] = {
                        'Latitude': catchment_data['Latitude'].iloc[0],
                        'Longitude': catchment_data['Longitude'].iloc[0],
                    }
                    for rp in return_periods:
                        combined_data[catchment_id][f'Stationary Return_{rp}'] = []
                        combined_data[catchment_id][f'Non Stationary Return_{rp}'] = []
                        combined_data[catchment_id][f'Difference Between Stationary and Non Stationary Return_{rp}'] = []

                for rp in return_periods:
                    combined_data[catchment_id][f'Stationary Return_{rp}'].extend(catchment_data[f'Stationary Return value for {rp}-year'].tolist())
                    combined_data[catchment_id][f'Non Stationary Return_{rp}'].extend(catchment_data[f'Non-Stationary Return value for {rp}-year'].tolist())
                    combined_data[catchment_id][f'Difference Between Stationary and Non Stationary Return_{rp}'].extend(catchment_data[f'Change in Precipitation for return period {rp}-year'].tolist())

    results = []
    for catchment_id, data in combined_data.items():
        result = {
            'Catchment ID': catchment_id,
            'Latitude': data['Latitude'],
            'Longitude': data['Longitude'],
        }
        for rp in return_periods:
            result[f'Stationary Return value for {rp}-year'] = sum(data[f'Stationary Return_{rp}']) / len(data[f'Stationary Return_{rp}'])
            result[f'Non Stationary Return value for {rp}-year'] = sum(data[f'Non Stationary Return_{rp}']) / len(data[f'Non Stationary Return_{rp}'])
            result[f'Difference Between Stationary and Non Stationary Return value for {rp}-year'] = sum(data[f'Difference Between Stationary and Non Stationary Return_{rp}']) / len(data[f'Difference Between Stationary and Non Stationary Return_{rp}'])

        results.append(result)

    result_df = pd.DataFrame(results)
    output_filename = os.path.join(output_directory, f"{ssp}_{period}.xlsx")
    result_df.to_excel(output_filename, index=False)
    print(f"Results saved to {output_filename}")

In [9]:
# Define the SSPs and periods
ssps = ["Historical", "ssp126", "ssp245", "ssp370", "ssp585"]
periods = ["1981-2010", "2041-2070", "2071-2100"]

# Process the files
directory = "/DATA1/ankit_new_data/Different Model Analysis"
output_directory = "/DATA1/ankit_new_data/Different Model Analysis/Combined Analysis"

for ssp in ssps:
    if ssp == "Historical":
        process_excel_files(directory, output_directory, ssp, periods[0])
    else:
        process_excel_files(directory, output_directory, ssp, periods[1])
        process_excel_files(directory, output_directory, ssp, periods[2])


Results saved to /DATA1/ankit_new_data/Different Model Analysis/Combined Analysis/Historical_1981-2010.xlsx
Results saved to /DATA1/ankit_new_data/Different Model Analysis/Combined Analysis/ssp126_2041-2070.xlsx
Results saved to /DATA1/ankit_new_data/Different Model Analysis/Combined Analysis/ssp126_2071-2100.xlsx
Results saved to /DATA1/ankit_new_data/Different Model Analysis/Combined Analysis/ssp245_2041-2070.xlsx
Results saved to /DATA1/ankit_new_data/Different Model Analysis/Combined Analysis/ssp245_2071-2100.xlsx
Results saved to /DATA1/ankit_new_data/Different Model Analysis/Combined Analysis/ssp370_2041-2070.xlsx
Results saved to /DATA1/ankit_new_data/Different Model Analysis/Combined Analysis/ssp370_2071-2100.xlsx
Results saved to /DATA1/ankit_new_data/Different Model Analysis/Combined Analysis/ssp585_2041-2070.xlsx
Results saved to /DATA1/ankit_new_data/Different Model Analysis/Combined Analysis/ssp585_2071-2100.xlsx
